<a href="https://colab.research.google.com/github/Amzilynn/Geoepidemiology-Profiling/blob/main/Geoepidemiology_dataexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load your datasets
modis_df = pd.read_csv('master_modis_all_regions.csv')
gpm_df = pd.read_csv('master_gpm_all_regions.csv')
climate_df = pd.read_csv('master_climate_2022_2024.csv')

#  Check the columns to decide the merge key
print(modis_df.columns)
print(gpm_df.columns)
print(climate_df.columns)

Index(['LST_Day_1km_max', 'LST_Day_1km_mean', 'LST_Day_1km_min',
       'LST_Night_1km_max', 'LST_Night_1km_mean', 'LST_Night_1km_min',
       'QC_Day_max', 'QC_Day_mean', 'QC_Day_min', 'QC_Night_max',
       'QC_Night_mean', 'QC_Night_min', 'date', 'region', 'satellite', 'year',
       'month', 'date_dt', 'temp_diff'],
      dtype='object')
Index(['date', 'precipitation_max', 'precipitation_mean', 'precipitation_min',
       'precipitation_sum', 'region', 'year', 'date_dt'],
      dtype='object')
Index(['region', 'year', 'date', 'date_dt', 'temperature_2m',
       'relative_humidity', 'dewpoint_2m', 'total_precipitation',
       'wind_speed_10m', 'soil_water_layer_1', 'soil_water_layer_2',
       'soil_water_layer_3'],
      dtype='object')


In [2]:
import pandas as pd

# 1. Load datasets
modis_df = pd.read_csv('master_modis_all_regions.csv')
gpm_df = pd.read_csv('master_gpm_all_regions.csv')
climate_df = pd.read_csv('master_climate_2022_2024.csv')

# 2. Ensure 'date' columns are datetime
modis_df['date'] = pd.to_datetime(modis_df['date'])
gpm_df['date'] = pd.to_datetime(gpm_df['date'])
climate_df['date'] = pd.to_datetime(climate_df['date'])

# 3. Merge MODIS + GPM daily
daily_merge = pd.merge(
    modis_df,
    gpm_df,
    on=['region', 'year', 'date'],
    how='outer',  # keep all days even if missing in one dataset
    suffixes=('_modis', '_gpm')
)

# 4. Merge with Climate data
daily_merge = pd.merge(
    daily_merge,
    climate_df,
    on=['region', 'year', 'date'],
    how='outer',
    suffixes=('', '_climate')
)

# 5. Save final daily dataset
daily_merge.to_csv('master_3_Test.csv', index=False)

print("Daily datasets merged successfully.")


Daily datasets merged successfully.


In [3]:
# Show the first 5 rows
print(daily_merge.head())

# Optional: show the first 10 rows
print(daily_merge.head(10))


   LST_Day_1km_max  LST_Day_1km_mean  LST_Day_1km_min  LST_Night_1km_max  \
0          15742.0      15135.674716          14859.0            15013.0   
1              NaN               NaN              NaN                NaN   
2              NaN               NaN              NaN                NaN   
3              NaN               NaN              NaN                NaN   
4              NaN               NaN              NaN                NaN   

   LST_Night_1km_mean  LST_Night_1km_min  QC_Day_max  QC_Day_mean  QC_Day_min  \
0        14814.252183            14443.0        81.0     2.930647         0.0   
1                 NaN                NaN         NaN          NaN         NaN   
2                 NaN                NaN         NaN          NaN         NaN   
3                 NaN                NaN         NaN          NaN         NaN   
4                 NaN                NaN         NaN          NaN         NaN   

   QC_Night_max  ...  date_dt_gpm     date_dt temperatur

In [4]:
daily_merge.isnull().sum()

,0
LST_Day_1km_max,4795
LST_Day_1km_mean,4795
LST_Day_1km_min,4795
LST_Night_1km_max,4809
LST_Night_1km_mean,4809
LST_Night_1km_min,4809
QC_Day_max,4795
QC_Day_mean,4795
QC_Day_min,4795
QC_Night_max,4795


In [7]:
daily_merge.shape

(5480, 33)

In [8]:
# Number of rows where MODIS is missing
missing_modis = daily_merge['LST_Day_1km_max'].isna().sum()
print("Missing MODIS rows:", missing_modis)

# Number of rows where GPM is missing
missing_gpm = daily_merge['precipitation_max'].isna().sum()
print("Missing GPM rows:", missing_gpm)

# Number of rows where Climate is missing
missing_climate = daily_merge['temperature_2m'].isna().sum()
print("Missing Climate rows:", missing_climate)


Missing MODIS rows: 4795
Missing GPM rows: 0
Missing Climate rows: 15


In [13]:
import pandas as pd
import numpy as np

# 1️⃣ Load your merged dataset
daily_merge = pd.read_csv('master_3_Test.csv')

# 2️⃣ Ensure 'date' column is datetime
daily_merge['date'] = pd.to_datetime(daily_merge['date'])

# 3️⃣ Sort by region and date
daily_merge = daily_merge.sort_values(['region', 'date'])

# 4️⃣ Columns to interpolate (MODIS LST)
modis_cols = [
    'LST_Day_1km_max', 'LST_Day_1km_mean', 'LST_Day_1km_min',
    'LST_Night_1km_max', 'LST_Night_1km_mean', 'LST_Night_1km_min'
]

# 5️⃣ Interpolate missing MODIS values per region using `transform`
daily_merge[modis_cols] = daily_merge.groupby('region')[modis_cols] \
    .transform(lambda group: group.interpolate(method='linear'))  # linear interpolation

# 6️⃣ Optional: fill any remaining NaNs at start/end using `transform`
daily_merge[modis_cols] = daily_merge.groupby('region')[modis_cols] \
    .transform(lambda group: group.fillna(method='bfill').fillna(method='ffill'))

# 7️⃣ Save the interpolated daily dataset
daily_merge.to_csv('master_3_Test.csv', index=False)

# 8️⃣ Quick check
print(daily_merge.head(10))
print(daily_merge[modis_cols].isna().sum())  # should be 0

/tmp/ipython-input-3962539528.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .transform(lambda group: group.fillna(method='bfill').fillna(method='ffill'))
/tmp/ipython-input-3962539528.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .transform(lambda group: group.fillna(method='bfill').fillna(method='ffill'))


   LST_Day_1km_max  LST_Day_1km_mean  LST_Day_1km_min  LST_Night_1km_max  \
0         15742.00      15135.674716        14859.000          15013.000   
1         15734.25      15134.682066        14770.125          15015.625   
2         15726.50      15133.689416        14681.250          15018.250   
3         15718.75      15132.696766        14592.375          15020.875   
4         15711.00      15131.704116        14503.500          15023.500   
5         15703.25      15130.711466        14414.625          15026.125   
6         15695.50      15129.718815        14325.750          15028.750   
7         15687.75      15128.726165        14236.875          15031.375   
8         15680.00      15127.733515        14148.000          15034.000   
9         15647.25      15111.455632        14136.125          15025.750   

   LST_Night_1km_mean  LST_Night_1km_min  QC_Day_max  QC_Day_mean  QC_Day_min  \
0        14814.252183          14443.000        81.0     2.930647         0.0   
1